In [1]:
from rdkit.Chem import AllChem
from rdkit import DataStructs
import rdkit.Chem as Chem

from pandas import Series,DataFrame
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.patches import *
from matplotlib.ticker import MultipleLocator, FormatStrFormatter 
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')

import seaborn as sns
import warnings

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, concatenate
from tensorflow.keras.losses import MeanSquaredError,MSE
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model

import spektral
from spektral.data import *
from spektral.datasets.mnist import MNIST
from spektral.layers import GCNConv,GlobalSumPool,ECCConv

import csv
import time
import umap 
import random
from sklearn.metrics import r2_score
from sklearn import linear_model
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('data/theoretical calculation data/data.csv',header=None )
data.columns = ['index','Adduct','SMILES','True CCS','ref','CCS Type','Class','Name'] 
data.drop_duplicates(subset = ['SMILES'],keep='first',inplace=True)
data

,index,Adduct,SMILES,True CCS,ref,CCS Type,Class,Name
0,0,[M+Na]+,CCn1nc(C(=O)O)c(=O)c2cc3c(cc21)OCO3,165.200000,7,TW,small molecule,CINOXACIN
1,1,[M+H]+,Cc1nccn1CC1CCc2c(c3cccc(O)c3n2C)C1=O,173.700000,13,TW,small molecule,Ondansetron-M 8-hydroxy
2,2,[M+H]+,C/C=C/c1ccc2c(c1)OCO2,138.400000,7,TW,small molecule,ISOSAFROLE
3,3,[M-H]-,CCCCCCCC/C=C\CCCCCCCCCCCCCCCCCCCC(=O)O,211.650000,21,TIMS,lipid,FA(30:1)
4,4,[M+H]+,C=CC(=O)O[C@H](COCCCCCCCCCCCCCCCC)COP(=O)([O-]...,235.600000,20,TIMS,lipid,PC(o19:1)
5,5,[M+H]+,Cn1c(=O)[nH]c2ncn(C)c2c1=O,134.812564,4,DT,small molecule,"1,7-Dimethylxanthine"
6,6,[M+H]+,CCCCCCCCCCCCCCCCC(=O)N[C@@H](CO)[C@H](O)CCCCCC...,258.600000,9,DT,lipid,Cer(35:0)
7,7,[M+Na]+,CC/C=C\C/C=C\C/C=C\C/C=C\C/C=C\C/C=C\CCC(=O)OC...,296.880000,1,DT,lipid,PS(44:10)
8,8,[M+H]+,OC(Cn1cncn1)(Cn1cncn1)c1ccc(F)cc1F,163.650000,13,TW,small molecule,Fluconazole
9,9,[M+Na]+,O=c1[nH]c(=O)c2ncn(C3OC(COP(=O)(O)O)C(O)C3O)c2...,176.600000,16,DT,small molecule,XANTHOSINE-MONOPHOSPHATE


In [3]:
GCN_smiles  = list(data['SMILES'])
GCN_adducts = list(data['Adduct'])
GCN_Index   = list(data['index'])
GCN_CCS     = list(data['True CCS'])
GCN_Class   = list(data['Class'])

In [4]:
smiles, ccs, adduct, Coordinate = [], [], [], []
for i in range(len(GCN_Index)):
    MOL = Chem.MolFromSmiles(GCN_smiles[i])
    atoms = [atom.GetSymbol() for atom in MOL.GetAtoms()]
    one_atom = []
    f = csv.reader(open('data/theoretical calculation data/Coordinate data/'+str(GCN_Index[i])+'.csv','r'))
    files = [i for i in f]
    for j in range(len(atoms)):
        one_atom.append([float(iii) for iii in files[j+1][2:]])
    Coordinate.append(one_atom)
    smiles.append([GCN_smiles[i]])
    ccs.append([GCN_CCS[i]])
    adduct.append(GCN_adducts[i])

Max_Coor =  15.615155868453662
Min_Coor = -15.657021258992941
for i in range(len(Coordinate)):
    Coordinate[i] = (np.array((Coordinate[i])) - Min_Coor) / (Max_Coor - Min_Coor)

Atom_radius = {'N' :71, 'Se':116, 'F':64, 'Co':111, 'O':63,'As':121,
               'Br':114,'Cl':99,  'S':103,'C' :75, 'P':111, 'I':133,'H':32}
Atom_radius_list = [Atom_radius[i] for i in Atom_radius]
Max_radius, Min_radius = np.max(Atom_radius_list), np.min(Atom_radius_list)
for i in Atom_radius:
    Atom_radius[i] = (Atom_radius[i] - Min_radius) / (Max_radius-Min_radius)
    
Atom_mass = {'N':14.00674,'Se':78.96,'F':18.9984032,'Co':58.933195,'As':74.92160,
             'O':15.9994,'Br':79.904,'Cl':35.453,'S':32.065,'C':12.0107,
             'P':30.973762,'I':126.90447,'H':1.00794}
Atom_mass_list = [Atom_mass[i] for i in Atom_mass]
Max_mass, Min_mass = np.max(Atom_mass_list), np.min(Atom_mass_list)
for i in Atom_mass:
    Atom_mass[i] = (Atom_mass[i] - Min_mass) / (Max_mass-Min_mass)

All_Atoms = ['As', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'Se']
print(All_Atoms)

def convertToGraph(smi_lst):
    adj,adj_norm, features, edge_features = [], [], [], []
    maxNumAtoms = 50 
    NodeNumFeatures, EdgeNumFeatures, INDEX = 0, 4, -1
    for smi in smi_lst:
        INDEX += 1
        iMol = Chem.MolFromSmiles(smi[0]) 
        maxNumAtoms = iMol.GetNumAtoms()  
        iAdjTmp = Chem.rdmolops.GetAdjacencyMatrix(iMol) 
        one_edge_features = edge_feature(iMol)
        edge_features.append(one_edge_features)
        iFeature = np.zeros((maxNumAtoms, NodeNumFeatures))
        iFeatureTmp = []
        for atom in iMol.GetAtoms():
            iFeatureTmp.append(atom_feature(atom,INDEX))
        features.append(np.array(iFeatureTmp))
        adj.append(iAdjTmp)
            
    features = np.asarray(features)
    edge_features = np.asarray(edge_features)
    return adj, features, edge_features

def atom_feature(atom,INDEX):
    return np.array(
        one_of_k_encoding_unk(atom.GetSymbol() ,All_Atoms) +
        one_of_k_encoding_unk(atom.GetDegree(), [0, 1, 2, 3, 4]) +
        [Atom_radius[atom.GetSymbol()],Atom_mass[atom.GetSymbol()]] +
        one_of_k_encoding_unk(atom.IsInRing(), [0, 1]) +
        list(Coordinate[INDEX][atom.GetIdx()])
    )

def one_of_k_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def edge_feature(iMol):
    iAdjTmp = Chem.rdmolops.GetAdjacencyMatrix(iMol)
    Edge_feature = []
    count = 0
    for bond in iMol.GetBonds():
        count += 1
        bond_feature = np.array(
            one_of_k_encoding_unk(bond.GetBondTypeAsDouble(),[1,1.5,2,3])
        )
        Edge_feature.append(bond_feature)
        Edge_feature.append(bond_feature)
    Edge_feature = np.array(Edge_feature)
    Edge_feature = Edge_feature.astype(np.float)

    return Edge_feature

adj, features, edge_features = convertToGraph(smiles)

['As', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'Se']


In [5]:
class MyDataset(Dataset):
    def __init__(self, features, adj, edge_features, ccs, **kwargs):
        self.features = features
        self.adj = adj
        self.edge_features = edge_features
        self.ccs = ccs
        super().__init__(**kwargs)
        
    def read(self):
        return [Graph(x = self.features[i], 
                      a = self.adj[i], 
                      e = self.edge_features[i],
                      y = float(self.ccs[i][0])) for i in range(len(self.adj))]
    
DataSet = MyDataset(features, adj, edge_features, ccs)
print(DataSet)
adduct_SET = ['[M+H]+', '[M+Na]+', '[M-H]-']
adduct_SET.sort()
print(adduct_SET)

dataset_te = DataSet
adduct_te  = adduct

MyDataset(n_graphs=10)
['[M+H]+', '[M+Na]+', '[M-H]-']


In [6]:
import spektral
ECC_model = load_model('model/model.h5',
                       custom_objects = {"ECCConv": spektral.layers.ECCConv,
                                         "GlobalSumPool": spektral.layers.GlobalSumPool})

ECC_model_layer = Model(inputs = ECC_model.input[1:],
                        outputs= ECC_model.get_layer('ECC3').output)
    
loader_te = BatchLoader(dataset_te,batch_size=1,epochs=1,shuffle=False)
mols_nodes_f = []
for batch in loader_te:
    inputs, target = batch
    # holds a vector of all the nodes of a numerator
    predictions = ECC_model_layer(inputs, training=False)
    result = predictions[0].numpy()
    mols_nodes_f.append(result)

In [7]:
LJ_Data = pd.read_csv('data/theoretical calculation data/LJ_data.csv')
element  = list(LJ_Data['elements'])
distance = list(LJ_Data['distance']) 
energy   = list(LJ_Data['energy'])    
elememt_2 = 'N'
for i in range(len(element)):
    if element[i] == elememt_2:
        elememt_2_distance = distance[i]
        elememt_2_energy   = energy[i]

Atom_radius = {'N' :71, 'Se':116, 'F':64, 'Co':111, 'O':63,'As':121,
               'Br':114,'Cl':99,  'S':103,'C' :75, 'P':111, 'I':133,'H':32}

Sample_N = 100 # Number of sampling points
def sample_spherical(npoints, S, ndim=3, offset=1):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    return vec * (Atom_radius[S]/100 + 0.71 + offset)

In [8]:
from tqdm import *
import time, random
import sys
p_bar = tqdm(range(len(GCN_smiles)), desc="Calculation", total=len(GCN_smiles), ncols=90, file=sys.stdout)

Energy_sum = []
All_CONTRI_Raw = []

for i, p_bar_i in zip(range(0,len(GCN_smiles)), p_bar):
    
    mol = Chem.MolFromSmiles(GCN_smiles[i]) # Read the smiles of the i-th molecule
    mol_H = Chem.AddHs(mol)
    atoms = [atom.GetSymbol() for atom in mol_H.GetAtoms()]    
    
    # Read the coordinate data of the i-th molecule
    f = csv.reader(open('data/theoretical calculation data/Coordinate data/'+str(GCN_Index[i])+'.csv','r'))
    files = [_i for _i in f]
    
    # Calculate the norm of i-th molecular vector (mols_nodes_f[i])
    contribs_raw = [np.sqrt(np.sum(mols_nodes_f[i][j])**2) for j in range(len(mols_nodes_f[i]))]
    
    AllChem.ComputeGasteigerCharges(mol_H)
    Charge_contribs = [round(mol_H.GetAtomWithIdx(i).GetDoubleProp('_GasteigerCharge'), 10) for i in range(mol_H.GetNumAtoms())]
    
    one_All_E = [] # Used to store the energy calculations for all atoms of the ith molecule
    
    All_atom = [j for j in range(len(mols_nodes_f[i]))] # Get the index of all atoms of the ith molecule
    
    for Atom in All_atom:
        zb = [float(iii) for iii in files[Atom+1][2:]] # Get the coordinates of the Atom-th atom of the i-th molecule
        xi, yi, zi = sample_spherical(Sample_N, S=atoms[Atom]) # Get the coordinates of all sampling points for the Atom-th atom of the ith molecule
        
        All_data = []
        # Calculate the energy of each sample point and place it in All_data
        # Iterate through each sample point
        for k in range(Sample_N):
            N_Coordinates = np.array([xi[k]+zb[0],yi[k]+zb[1],zi[k]+zb[2]]) # Coordinates of the k-th sampling point
            E1, E2 = [], [] 
            flag = 0
            # Sampling point to energy per atom
            for Atom2 in range(len(atoms)):
                Coordinates = [float(iii) for iii in files[Atom2+1][2:]]
                dis = np.sqrt(np.sum((np.array(Coordinates)-N_Coordinates)**2))
                
                # If this sampling point is also very close to other atoms, it is unreasonable
                if dis <= (Atom_radius[atoms[Atom2]]/100 + 0.71 + 0):
                    flag = 1
                    break
                    
                elememt_1 = atoms[Atom2]
                for ii in range(len(element)):
                    if element[ii] == elememt_1:
                        elememt_1_distance =  distance[ii]
                        elememt_1_energy   = energy[ii]
                        break
                        
                Distance_constant = ((elememt_1_distance + elememt_2_distance)/2) / (2**(1/6))
                Energy_constant   = np.sqrt(elememt_1_energy * elememt_2_energy) * 0.0433641
                
                Fisrt = 4*Energy_constant*((Distance_constant/dis)**12 - (Distance_constant/dis)**6)
                Secen = Charge_contribs[Atom2]*np.array([Coordinates[0] / dis**3,Coordinates[1] / dis**3,Coordinates[2] / dis**3])
                
                E1.append(Fisrt)
                E2.append(Secen)
            
            if flag == 1:
                continue
            # Calculate the total potential energy of a sample point in the molecule
            All_data.append(np.sum(E1)+(((1.60217e-19)**2)*1.73968851e-30)/(8*np.pi)*np.sum(np.sum(E2,axis = 0)**2))
            
        # Record the average value of all sampling points
        one_All_E.append(float(np.mean(All_data)))
    
    one_All_contribs_raw = [contribs_raw[i] for i in All_atom]
    
    Energy_sum.append(np.sum(one_All_E))
    All_CONTRI_Raw.append(np.sum(one_All_contribs_raw))    
p_bar.close()

Calculation:  90%|████████████████████████████████████▉    | 9/10 [00:29<00:03,  3.31s/it]


In [9]:
data = {'All atoms of a molecule sum energy' : Energy_sum,
        '3*ECC-nodes sum contr (norm)' : All_CONTRI_Raw,}
df = DataFrame(data)
df.corr()

,All atoms of a molecule sum energy,3*ECC-nodes sum contr (norm)
All atoms of a molecule sum energy,1.000000,0.912125
3*ECC-nodes sum contr (norm),0.912125,1.000000
